### Train a Noise2Void model

Both the CARE network and Noise2Noise network you trained in part 1 and 2 require that you acquire additional data for the purpose of denoising. For CARE we used a paired acquisition with high SNR, for Noise2Noise we had paired noisy acquisitions. We will now train a Noise2Void network from single noisy images.

This notebook uses a single image from the SEM data from the Noise2Noise notebook, but as you'll see in Task 3.1 if you brought your own raw data you should adapt the notebook to use that instead.

We use the [Careamics](https://careamics.github.io) library

<div class="alert alert-block alert-danger\"> Set your kernel to 
</div>




<div class="alert alert-block alert-info"><h3>Task 2.2: Bla</h3>
    
This notebook uses a single image from the SEM data from the Noise2Noise notebook.

If you brought your own raw data, use that instead! The only requirement is that the noise in your data is pixel-independent and zero-mean. If you're unsure whether your data fulfills that requirement or you don't yet understand why it is necessary ask one of us to discuss!

If you don't have suitable data of your own, feel free to find some online or ask your fellow course participants. You can however also stick with the SEM data provided here and compare the results to what you achieved with Noise2Noise in the previous part.


</div>

![diff_costs](static/diff_costs.png)

<hr style="height:2px;"><div class="alert alert-block alert-success"><h1>Checkpoint 1</h1>

</div>

In [ ]:
import shutil
from pathlib import Path

import matplotlib.pyplot as plt
import tifffile
from careamics_portfolio import PortfolioManager

from careamics import CAREamist
from careamics.config import (
    create_n2v_training_configuration,
)

### Part 1. Prepare the data

In [ ]:
# Explore portfolio
portfolio = PortfolioManager()
print(portfolio.denoising)

In [ ]:
# Download files # TODO File should be reused from previous exercise
root_path = Path("./data")
files = portfolio.denoising.N2V_SEM.download(root_path)
print(f"List of downloaded files: {files}")

#### Visualize training data

In [ ]:
# Load images
train_image = tifffile.imread(files[0])
print(f"Train image shape: {train_image.shape}")
plt.imshow(train_image, cmap="gray")

#### Visualize validation data

In [ ]:
val_image = tifffile.imread(files[1])
print(f"Validation image shape: {val_image.shape}")
plt.imshow(val_image, cmap="gray")

In [ ]:
# Set paths

data_path = Path(root_path / "n2v_sem")
train_path = data_path / "train"
val_path = data_path / "val"

train_path.mkdir(parents=True, exist_ok=True)
val_path.mkdir(parents=True, exist_ok=True)

shutil.copy(root_path / files[0], train_path / "train_image.tif")
shutil.copy(root_path / files[1], val_path / "val_image.tif")

### Part 2. Create configuraion

In [ ]:
training_config = create_n2v_training_configuration(
    experiment_name="LevitatingFrog",
    data_type="tiff",
    axes="YX",
    patch_size=[64, 64],
    batch_size=128,
    num_epochs=10,
)

#### Initialize the Model

Create a Pytorch Lightning module

Please take as look at the [documentation](https://careamics.github.io) to see the full list of parameters and configuration options

In [ ]:
engine = CAREamist(source=training_config)

### Part 3. Run training 

We need to specify the paths to training and validation data

In [ ]:
engine.train(path_to_train_data=train_path, path_to_val_data=val_path)

### Part 4. Prediction

In [ ]:
preds = engine.predict(source=train_path, tile_size=(256, 256))

### Visualize predictions